## Setup

In [1]:
import sys
import os
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time

import numpy as np
import pandas as pd
import isuelogit as isl
import glob

In [2]:
# Path management
main_dir = str(Path(os.path.abspath('')).parents[1])
os.chdir(main_dir)
print('main dir:', main_dir)

sys.path.append(os.path.join(main_dir, 'src'))

isl.config.dirs['read_network_data'] = "input/network-data/fresno/"

main dir: /Users/pablo/github/pesuelogit


In [3]:
# Internal modules
from pesuelogit.visualizations import plot_predictive_performance, plot_convergence_estimates
from pesuelogit.etl import data_curation, add_period_id

## Read spatiotemporal data

In [ ]:
folderpath = isl.config.dirs['read_network_data'] + 'links/spatiotemporal-data/'
df = pd.concat([pd.read_csv(file) for file in glob.glob(folderpath + "*link-data*")], axis=0)

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df[df['date'].dt.dayofweek.between(0, 3)]
# df = df[df['date'].dt.year == 2019]

df['period'] = df['date'].astype(str) + '-' + df['hour'].astype(str)
df['period'] = df.period.map(hash)

In [ ]:
# Add period id for time-varying estimation

period_feature = 'hour'

df['period'] = df['date'].astype(str) + '-' + df[period_feature].astype(str)
# df['period'] = df.period.map(hash)

df = add_period_id(df, period_feature='hour')

period_keys = df[[period_feature,'period_id']].drop_duplicates().reset_index().drop('index',axis =1).sort_values('hour')
print(period_keys)

## Data curation

In [ ]:
df['tt_ff'] = np.where(df['link_type'] != 'LWRLK', 0,df['length']/df['speed_ref_avg'])
df.loc[(df.link_type == "LWRLK") & (df.speed_ref_avg == 0),'tt_ff'] = float('nan')

df['tt_avg'] = np.where(df['link_type'] != 'LWRLK', 0,df['length']/df['speed_hist_avg'])
df.loc[(df.link_type == "LWRLK") & (df.speed_hist_avg == 0),'tt_avg'] = float('nan')

df = data_curation(df)

In [ ]:
df[['speed_ref_avg','speed_hist_avg', 'tt_ff', 'tt_avg']].describe()

## Data processing

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df.date.dt.year

In [ ]:
df.query('year == 2019')[['counts', 'tt_ff', 'tt_avg', 'tf_inrix', 'speed_avg']].describe()

In [ ]:
df.query('year == 2020')[['counts', 'tt_ff', 'tt_avg', 'tf_inrix', 'speed_avg']].describe()

In [ ]:
print(1-1793.271103/1865.514775, 1-17.469430/18.895705)


## EDA

### Observations

In [ ]:
# Single period models
df.loc[(df.hour == 16) & (df.year == 2019),['counts', 'tt_avg']].describe()

In [ ]:
# TVODLULPE
df.loc[(df.hour.isin([6,7,8,15,16,17])) & (df.year == 2019),['counts', 'tt_avg']].describe()

### Link attributes

In [ ]:
features_Z = ['tt_sd', 'median_inc', 'incidents', 'bus_stops', 'intersections']

In [ ]:
features_plot = ['speed_sd', 'median_inc', 'incidents', 'bus_stops', 'intersections']

sns.lineplot(x= 'date', y = 'value', hue = 'variable', data =pd.melt(df.groupby('date')[features_plot].mean().reset_index(),id_vars= ['date']))
plt.tight_layout()
plt.legend(title="")
plt.ylabel("value of exogenous feature")
plt.xticks(rotation=90)
plt.show()

## Analyze single file of inrix data

In [ ]:
inrix_df = pd.read_csv(f"{os.getcwd()}/input/private/inrix/2020-10-01.csv")
inrix_df['ts'] = pd.to_datetime(inrix_df['UTC Date Time'])
inrix_df['hour'] = inrix_df.ts.dt.hour
inrix_df['speed'] = inrix_df['Speed(km/hour)']*0.62137119223

In [ ]:
# Select data from same time range
inrix_df = inrix_df[inrix_df.hour.isin(range(4,23))]

In [ ]:
inrix_df.head()

In [ ]:
inrix_df.describe()

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (10,4))

sns.lineplot(data = inrix_df.groupby('hour')[['Speed(km/hour)']].mean(),
             ax = axs[0])
axs[0].set_title('avg of speed')

sns.lineplot(data = inrix_df.groupby('hour')[['speed']].std(),
             ax = axs[1])
axs[1].set_title('std of speed');

In [ ]:
# Filter only data from inrix segments that were matched with the network links

fig, axs = plt.subplots(1,2, figsize = (10,4))

sns.lineplot(data = inrix_df[inrix_df['Segment ID'].isin(list(df.inrix_id.dropna().unique()))].groupby('hour')[['speed']].mean(),
             ax = axs[0])
axs[0].set_title('avg of speed')

sns.lineplot(data = inrix_df[inrix_df['Segment ID'].isin(list(df.inrix_id.dropna().unique()))].groupby('hour')[['speed']].std(),
             ax = axs[1])
axs[1].set_title('std of speed');

## Stats by date

In [ ]:
# To check that there is a balanced amount of observations per date
obs_date = df.groupby('date')['hour'].count()

In [ ]:
# Stats by date
df.groupby('date')[['speed_sd','speed_avg', 'counts']].mean().assign(total_obs = obs_date)

### Cross sectional

In [ ]:
eda_df = df[df.year == 2019].copy()
#eda_df = df.copy()

eda_df['day'] = eda_df.date.dt.day#.astype(str)
eda_df['hour_id'] = eda_df['hour'].astype(str).apply(lambda x: time.strftime("%l%p", time.strptime(x,"%H")))
eda_df['date'] = eda_df['date'].astype(str)

# Transform to monthly income
eda_df['median_inc'] = eda_df['median_inc']/12

In [ ]:
eda_df['hour_id'] = pd.Categorical(eda_df['hour_id'], categories = eda_df[['hour_id', 'hour']].drop_duplicates().sort_values('hour')['hour_id'], ordered = True)

eda_df['day'] = pd.Categorical(eda_df['day'], categories = eda_df[['day']].drop_duplicates().sort_values('day')['day'], ordered = True)


In [ ]:
eda_df

In [ ]:
eda_df.groupby('day')[['counts']].mean().reset_index()

In [ ]:
daily_counts = eda_df.groupby('day')[['counts']].mean().reset_index()
daily_counts['day'] = daily_counts['day'].astype(str)

daily_speed = eda_df.groupby('day')[['speed_avg']].mean().reset_index()
daily_speed['day']  = daily_speed['day'].astype(str)

daily_attributes = pd.melt(eda_df.groupby('day')[features_plot].mean().reset_index(),id_vars= ['day'])
daily_attributes['day']  = daily_attributes['day'].astype(str)

In [ ]:
daily_attributes['day']

In [ ]:
#formatter = matplotlib.days.dayFormatter('%d')
#locator = matplotlib.dates.DayLocator(interval = 2)

fig, axs = plt.subplots(1,3, figsize = (12,4))

sns.lineplot(x= 'day', y = 'counts', data = daily_counts,
             ax = axs[0])

sns.lineplot(x= 'day', y = 'speed_avg', data =daily_speed,
             ax = axs[1])

sns.lineplot(x= 'day', y = 'value', hue = 'variable', data =daily_attributes,
             ax = axs[2])

for ax in axs:
#     ax.set_xticks(np.arange(1,31,2))
#     ax.set_xticklabels(np.arange(1,31,2))
    #ax.xaxis.set_major_formatter(formatter)
    #ax.xaxis.set_major_locator(locator)
    ax.set_xlabel('day of the month')

axs[0].set_ylabel('link flow [vehicles per hour]')
axs[1].set_ylabel('speed [miles per hour]')

axs[2].legend(title="", loc = 'upper right')
axs[2].set_ylabel("value of exogenous attribute")

#list(map(lambda x: x.set_xticklabels(x.get_xticks(), rotation=0), axs));
#fig.autofmt_xdate(rotation = 90, ha = 'center')
fig.tight_layout()

### By hour of day

In [ ]:
link_keys = eda_df[(eda_df.counts>0) & (eda_df.speed_avg>0)].link_key.unique()
link_keys = link_keys[0:10]

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (10,4))

sns.lineplot(x= 'hour', y = 'counts', hue = 'link_key',
             data =eda_df[eda_df.link_key.isin(link_keys)].groupby(['hour','link_key'])[['counts']].mean().reset_index(),
             ax = axs[0])

sns.lineplot(x= 'hour', y = 'speed_avg', hue = 'link_key',
             data =eda_df[eda_df.link_key.isin(link_keys)].groupby(['hour','link_key'])[['speed_avg']].mean().reset_index(),
             ax = axs[1])

In [ ]:
# Analysis for links where link counts are reported

fig, axs = plt.subplots(1,4, figsize = (20,4))

sns.lineplot(x= 'hour', y = 'counts', data =eda_df.groupby(['hour'])[['counts']].mean().reset_index(),
             ax = axs[0])

sns.lineplot(x= 'hour', y = 'speed_avg',
             data =eda_df[~eda_df.counts.isna()].groupby(['hour'])[['speed_avg']].mean().reset_index(),
             ax = axs[1])

sns.lineplot(x= 'hour', y = 'speed_max',
             data =eda_df[~eda_df.counts.isna()].groupby(['hour'])[['speed_max']].max().reset_index(),
             ax = axs[2])

sns.lineplot(x= 'hour', y = 'speed_sd',
             data =eda_df[~eda_df.counts.isna()].groupby(['hour'])[['speed_sd']].mean().reset_index(),
             ax = axs[3])

plt.show()

In [ ]:
# Select data from 2019 only
eda_df = eda_df[eda_df.year == 2019]
eda_df['hour_id'] = pd.Categorical(eda_df['hour_id'], categories = eda_df[['hour_id', 'hour']].drop_duplicates().sort_values('hour')['hour_id'], ordered = True)

In [ ]:
# Analysis for links where link counts are reported

fig, axs = plt.subplots(1,2, figsize = (9,4))

sns.lineplot(x= 'hour_id', y = 'counts', data =eda_df.groupby(['hour_id'])[['counts']].mean().reset_index(),
             ax = axs[0])

sns.lineplot(x= 'hour_id', y = 'speed_avg',
             data =eda_df[~eda_df.counts.isna()].groupby(['hour_id'])[['speed_avg']].mean().reset_index(),
             ax = axs[1])
fig.autofmt_xdate(rotation = 90, ha = 'center')

axs[0].set_ylabel('average link flow [vehicles per hour]')
axs[1].set_ylabel('average speed [miles per hour]')
list(map(lambda x: x.set_xlabel('hour'),axs))
plt.show()

## Read models results

In [ ]:
ts = 230211161720
train_results_dfs = pd.read_csv(f'output/tables/{ts}_train_results_Fresno.csv', index_col = [0])
test_results_dfs = pd.read_csv(f'output/tables/{ts}_train_results_Fresno.csv', index_col = [0])

## Configuration

In [ ]:
_XTICKS_SPACING = 50

In [ ]:
train_results_dfs

### Model 1: Estimation of utility function (LUE)

In [ ]:
print('\nLUE: Estimation of utility function')

# Average reliability ratio over epochs
plot_convergence_estimates(estimates=train_results_dfs[train_results_dfs.model == 'lue'].\
                   assign(rr = train_results_dfs[train_results_dfs.model == 'lue']['tt_sd']/train_results_dfs[train_results_dfs.model == 'lue']['tt'])[['epoch','rr']],
                       xticks_spacing = _XTICKS_SPACING)

plot_predictive_performance(train_losses=train_results_dfs[train_results_dfs.model == 'lue'],
                            val_losses=test_results_dfs[test_results_dfs.model == 'lue'],
                            xticks_spacing = _XTICKS_SPACING)

# print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(lue.theta.numpy())))}")
print(f"alpha = {list(train_results_dfs[train_results_dfs.model == 'lue']['alpha'])[-1]: 0.2f}, "
      f"beta  = {list(train_results_dfs[train_results_dfs.model == 'lue']['beta'])[-1]: 0.2f}")
# print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(lue.q - fresno_network.q.flatten())): 0.2f}")
# print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

### Model 2: OD + utility estimation with historic OD (ODLUE)

In [ ]:
print('\nODLUE: OD + utility estimation with historic OD')

train_results_dfs.loc[train_results_dfs.model == 'odlue','model'] = 'odlue'
test_results_dfs.loc[test_results_dfs.model == 'odlue','model'] = 'odlue'

# Average reliability ratio over epochs
plot_convergence_estimates(estimates=train_results_dfs[train_results_dfs.model == 'odlue'].\
                   assign(rr = train_results_dfs[train_results_dfs.model == 'odlue']['tt_sd']/train_results_dfs[train_results_dfs.model == 'odlue']['tt'])[['epoch','rr']],
                       xticks_spacing = _XTICKS_SPACING)

plot_predictive_performance(train_losses=train_results_dfs[train_results_dfs.model == 'odlue'],
                            val_losses=test_results_dfs[test_results_dfs.model == 'odlue'],
                            show_validation= False,
                            xticks_spacing = _XTICKS_SPACING)


# print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(odlue_1.theta.numpy())))}")
print(f"alpha = {list(train_results_dfs[train_results_dfs.model == 'odlue']['alpha'])[-1]: 0.2f}, "
      f"beta  = {list(train_results_dfs[train_results_dfs.model == 'odlue']['beta'])[-1]: 0.2f}")
# print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(odlue_1.q - fresno_network.q.flatten())): 0.2f}")
# print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

### Model 3: ODLUE + link specific performance parameters (ODLULPE)

In [ ]:
print('\nODLULPE: ODLUE + link performance parameters with historic OD matrix (link specifics alphas and betas)')

train_results_dfs.loc[train_results_dfs.model == 'odlulpe','model'] = 'odlulpe'
test_results_dfs.loc[test_results_dfs.model == 'odlulpe','model'] = 'odlulpe'

# Average reliability ratio over epochs
plot_convergence_estimates(estimates=train_results_dfs[train_results_dfs.model == 'odlulpe'].\
                   assign(rr = train_results_dfs[train_results_dfs.model == 'odlulpe']['tt_sd']/train_results_dfs[train_results_dfs.model == 'odlulpe']['tt'])[['epoch','rr']],
                       xticks_spacing = _XTICKS_SPACING)

plot_predictive_performance(train_losses=train_results_dfs[train_results_dfs.model == 'odlulpe'],
                            val_losses=test_results_dfs[test_results_dfs.model == 'odlulpe'],
                            show_validation= False,
                            xticks_spacing = _XTICKS_SPACING)

plot_convergence_estimates(
    estimates=train_results_dfs[train_results_dfs.model == 'odlulpe'][['epoch','alpha','beta']],
    xticks_spacing = _XTICKS_SPACING)

# print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(odlulpe_1.theta.numpy())))}")
print(f"alpha = {list(train_results_dfs[train_results_dfs.model == 'odlulpe']['alpha'])[-1]: 0.2f}, "
      f"beta  = {list(train_results_dfs[train_results_dfs.model == 'odlulpe']['beta'])[-1]: 0.2f}")
# print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(odlulpe_1.q - fresno_network.q.flatten())): 0.2f}")
# print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

### Model 4: ODLULPE with Time Varying OD and Utility Function (TVODLULPE)

In [ ]:
print('\ntvodlulpe: Time specific utility and OD, link performance parameters')

train_results_dfs.loc[train_results_dfs.model == 'tvodlulpe','model'] = 'tvodlulpe'
test_results_dfs.loc[test_results_dfs.model == 'tvodlulpe','model'] = 'tvodlulpe'

# Average reliability ratio over epochs
plot_convergence_estimates(estimates=train_results_dfs[train_results_dfs.model == 'tvodlulpe'].\
                   assign(rr = train_results_dfs[train_results_dfs.model == 'tvodlulpe']['tt_sd']/train_results_dfs[train_results_dfs.model == 'tvodlulpe']['tt'])[['epoch','rr']],
                       xticks_spacing = _XTICKS_SPACING)

plot_predictive_performance(train_losses=train_results_dfs[train_results_dfs.model == 'tvodlulpe'],
                            val_losses=test_results_dfs[test_results_dfs.model == 'tvodlulpe'],
                            show_validation= False,
                            xticks_spacing = _XTICKS_SPACING)

plot_convergence_estimates(
    estimates=train_results_dfs[train_results_dfs.model == 'tvodlulpe'][['epoch','alpha','beta']],
    xticks_spacing = _XTICKS_SPACING)

# print(f"theta = {dict(zip(utility_parameters.true_values.keys(), list(tvodlulpe_1.theta.numpy())))}")
print(f"alpha = {list(train_results_dfs[train_results_dfs.model == 'tvodlulpe']['alpha'])[-1]: 0.2f}, "
      f"beta  = {list(train_results_dfs[train_results_dfs.model == 'tvodlulpe']['beta'])[-1]: 0.2f}")
# print(f"Avg abs diff of observed and estimated OD: {np.mean(np.abs(tvodlulpe_1.q - fresno_network.q.flatten())): 0.2f}")
# print(f"Avg observed OD: {np.mean(np.abs(fresno_network.q.flatten())): 0.2f}")

## Summary of parameters estimates

In [ ]:
_EPOCHS = {'learning': 150, 'equilibrium': 50}

In [ ]:
# train_results_dfs = train_results_dfs[train_results_dfs['epoch'] <= _EPOCHS['learning']]

In [ ]:
train_results_dfs

In [ ]:
models = train_results_dfs.model.unique()

In [ ]:
results = pd.DataFrame({'parameter': [], 'model': []})

for model in models:
    results_model = train_results_dfs.loc[train_results_dfs.model == model].iloc[-1]
    results = results.append(pd.DataFrame(
        {'parameter': ['tt'] + features_Z +
                      ['fixed_effect_mean','fixed_effect_std',
                       'alpha_mean', 'alpha_std',
                       'beta_mean', 'beta_std',
                       # 'od_mean', 'od_std'
                       ],
         'values': list(results_model[['tt'] + features_Z]) +
                   [np.mean(results_model['fixed_effect']),np.std(results_model['fixed_effect']),
                    np.mean(results_model['alpha']),np.std(results_model['alpha']),
                    np.mean(results_model['beta']),np.std(results_model['beta']),
                    # np.mean(model.q),np.std(model.q)
                    ]
         }
    ).assign(model = model))

In [ ]:
results.pivot_table(index = ['parameter'], columns = 'model', values = 'values', sort=False).round(4)

In [ ]:
train_estimates = {}
train_losses = {}

for model in models:
    train_estimates[model] = train_results_dfs[train_results_dfs.model == model]

    train_estimates[model]['model'] = model

train_estimates_df = pd.concat(train_estimates.values())

train_estimates_df['rr'] = train_estimates_df['tt_sd']/train_estimates_df['tt']

estimates = train_estimates_df[['epoch','model','rr']].reset_index().drop('index',axis = 1)
#estimates = estimates[estimates.epoch != 0]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

g = sns.lineplot(data=estimates, x='epoch', hue='model', y='rr')

ax.set_ylabel('average reliability ratio')

plt.ylim(ymin=0)

ax.set_xticks(np.arange(estimates['epoch'].min(), estimates['epoch'].max() + 1, _XTICKS_SPACING))

plt.show()